# Python Dictionaries; A very clever data structure!

In [1]:
talk = {
    "title": "Python Dictionaries, A very clever data structure!",
    "author": "Kai Striega"
}

In [2]:
from copy import deepcopy
from dataclasses import dataclass
from itertools import islice
from sys import getsizeof
from typing import Hashable, Any

* Dictionaries play an extremely important role in Python
* Used in almost every python program
* The Python interpreter requires dictionaries to run Python code
* Important to know what the datastructures you use everyday do

## What is a dictionary
* A dictionary is a datastructure that maps a ``key`` to a ``value``
* The main operations on a dictionary are storing a value with some key and extracting the value given the key
* The equivalent datastructure is often known as a _hashmap_
* First published in 1953!

### Interface
* A dictionary is an _interface_
* It defines how an object should behave given different operations

### Defined Operations

* There are many operations that can be performed on dictionaries
* For the sake of brevity our dictionary will be defined as having three main operations:
    1. Insert new items ``d[k] = v``
    2. Lookup the value of a give key ``d[k]``
    3. Delete a (key, value) pair from the dictionary ``del d[k]``
* The full list is available in the [docs](https://docs.python.org/3/library/stdtypes.html#dict)

In [52]:
# ``d[k] = v`` adds the ``(k, v)`` pair to the dictionary
talk["audience"] = "PythonWA"
# Our dict now contains an ``audience`` key and ``Pythonistas`` value
talk

{'author': 'Kai Striega', 'audience': 'PythonWA'}

In [4]:
# ``d[k]`` returns the value assoicated with ``k``
talk["author"]

'Kai Striega'

In [5]:
# ``del d[k]`` delets the value associated with ``k``
del talk["title"]
talk

{'author': 'Kai Striega', 'audience': 'Pythonistas'}

# Implementation

## Overview
* There are several ways to implement this interface:
    1. A [Hashmap/Hash table](https://en.wikipedia.org/wiki/Hash_table)
    2. A [Linked List](https://en.wikipedia.org/wiki/Linked_list) with linear search for the elements
    3. A [Search Tree](https://en.wikipedia.org/wiki/Search_tree)
* Python chooses to implement this using a hashmap.
* A computer doesn't understand complex data structures such as dictionaries, lists or tuples
* All of these complex data structures have to be written by someone
* In the rest of this talk we will implement a hashmap from scratch
* CPython does this in [~5000 lines of C](https://github.com/python/cpython/blob/3.12/Objects/dictobject.c)
* We'll be working in Python using some abstractions

### Warning!

* These slides implement a hashmap using C idioms
* This is to help focus on one specific thing at a time
* And to fit everything onto slides
* **This is not Pythonic**

## Storing data

* The hashmap we will need to acquire memory
* As we haven't put anything into it yet, let's represent it as a list of ``None``s
* I've chosen to store 8 ``None`` objects as an example
* This is a very small amount of memory.

In [6]:
# Mimick 8 "buckets" in memory
dictionary = [None] * 8
dictionary

[None, None, None, None, None, None, None, None]

## Hashing

* We have some memory to work with, and we can access our memory using an index.
* But that isn't what we want to do. We want to be able to use some key to access the underlying value.
* To do this we will use what's called a [hash function](https://en.wikipedia.org/wiki/Hash_function).

* A **hash function** takes arbitrarily large data and converts it to an integer
* We can then interpret that bit pattern as an integer and use that integer as the index
* Designing good hash functions is difficult (much more on that later)
* Luckily Python has an inbuilt hash function - [hash](https://docs.python.org/3/library/functions.html#hash).

Let's play around with hash for a bit!

In [7]:
# Strings can be converted to numbers!
hash("Python")

3545465326777776709

In [8]:
# Numbers usually hash to themselves
hash(10)

10

In [9]:
# More complex data types can also be hashed
hash((10, "Python"))

-3297780909026427841

* These integers are far too large to fit into our small hashtable
* We can take the modulus of the integer to get our index
* This way we will always find a way to store the given key

In [10]:
hash("PythonWA") % 8

0

## Wrapping it all together

In [11]:
def dict_set(buckets, key, value):
    h = hash(key)
    i = h % len(buckets)
    buckets[i] = value

In [12]:
def dict_get(buckets, key):
    h = hash(key)
    i = h % len(dictionary)
    return buckets[i]

In [13]:
def dict_del(buckets, key):
    h = hash(key)
    i = h % len(dictionary)
    buckets[i] = None

In [14]:
toy_dict = [None] * 8
dict_set(toy_dict, 0, "Python")
dict_set(toy_dict, 1, "is awesome")
dict_set(toy_dict, 2, "!")
toy_dict

['Python', 'is awesome', '!', None, None, None, None, None]

In [15]:
dict_get(toy_dict, 2)

'!'

In [16]:
dict_del(toy_dict, 2)

In [17]:
toy_dict

['Python', 'is awesome', None, None, None, None, None, None]

## Hash Collisions

In [18]:
dict_set(toy_dict, 8, "Ben")

### What's happened here?

In [19]:
f"{hash(0) = }, {hash(8) = }"

'hash(0) = 0, hash(8) = 8'

In [20]:
f"{0 % 8 = }, {8 % 8 = }"

'0 % 8 = 0, 8 % 8 = 0'

* We've overwritten the existing data in our dictionary.
* This is called a _collision_.
* There are two common ways of resolving collisions:
    * Separate Chaining
    * Open Addressing

### Separate Chaining

* Build a [linked list](https://en.wikipedia.org/wiki/Linked_list) for each non-``None`` bucket.

![](../images/chaining_rotated.JPEG)

### Open Addressing

* Instead of generating an index using our hash we generate a _sequence_ of indecies
* This process is called _probing_ and the sequence we generate is called a _probe sequence_
* To add a new item to our dictionary we check each index generated by the probe sequence and assign it to the first empty slot.

![](../images/wasted_space_rotated.JPEG)

#### Linear Probing
$$ probes[i] = (hash(key) + i) \mathbin{\%} \text{number of buckets}$$

#### Quadratic Probing
$$ probes[i] = (hash(key) + a * i + b * i^2) \mathbin{\%} \text{number of buckets} $$

#### Pseudo-Random Probing
$$ probes[i] = (a * probes[i-1] + c) \mathbin{\%} \text{number of buckets} $$

### What does Python do?

* Very thorough [docs](https://github.com/python/cpython/blob/3.12/Objects/dictobject.c#L143)

In [21]:
PERTURB_SHIFT = 5

def probes(hash_value, hash_table_size):
    mask = hash_table_size - 1 # used to take modulus fast
    perturb = hash_value # used to perturb the probe sequence
    probe = hash_value & mask

    while True:
        yield probe

        perturb >>= PERTURB_SHIFT
        probe = (probe * 5 + perturb + 1) & mask

In [22]:
def dict_set(buckets, key, value):
    initial_hash = hash(key)
    size = len(buckets)
    for probe in probes(initial_hash, size):
        if buckets[probe] is None:
            buckets[probe] = value
            break

In [23]:
toy_dict = [None] * 8
dict_set(toy_dict, 0, "Python")
dict_set(toy_dict, 1, "is awesome")
dict_set(toy_dict, 2, "!")
toy_dict

['Python', 'is awesome', '!', None, None, None, None, None]

In [24]:
dict_set(toy_dict, 8, "Ben")
toy_dict

['Python', 'is awesome', '!', None, None, None, 'Ben', None]

### What about retrieving values?

In [25]:
def dict_get(buckets, key):
    # This method is __wrong__, can you spot the error?
    initial_hash = hash(key)
    size = len(buckets)
    for probe in probes(initial_hash, size):
        if buckets[probe]:
            return buckets[probe]
        else:
            raise KeyError(key)

In [26]:
toy_dict = [None] * 8
dict_set(toy_dict, 0, "Python")
dict_set(toy_dict, 1, "is awesome")
dict_set(toy_dict, 2, "!")
dict_set(toy_dict, 8, "Ben")
toy_dict

['Python', 'is awesome', '!', None, None, None, 'Ben', None]

In [27]:
dict_get(toy_dict, 8)

'Python'

#### Getting values correctly

In [28]:
@dataclass
class Bucket:
    key_hash: int
    key: Hashable
    value: Any

In [29]:
def dict_set(buckets, key, value):
    initial_hash = hash(key)
    size = len(buckets)
    bucket = Bucket(initial_hash, key, value)
    for probe in probes(initial_hash, size):
        if buckets[probe] is None:
            buckets[probe] = bucket
            break

In [30]:
def dict_get(buckets, key):
    initial_hash = hash(key)
    size = len(buckets)
    for probe in probes(initial_hash, size):
        bucket = buckets[probe]
        if bucket is None:
            raise KeyError(key)
        elif bucket.key_hash == initial_hash and bucket.key == key:
            return bucket.value

In [31]:
toy_dict = [None] * 8
dict_set(toy_dict, 0, "Python")
dict_set(toy_dict, 1, "is awesome")
dict_set(toy_dict, 2, "!")
dict_set(toy_dict, 8, "Ben")
toy_dict

[Bucket(key_hash=0, key=0, value='Python'),
 Bucket(key_hash=1, key=1, value='is awesome'),
 Bucket(key_hash=2, key=2, value='!'),
 None,
 None,
 None,
 Bucket(key_hash=8, key=8, value='Ben'),
 None]

In [32]:
dict_get(toy_dict, 8)

'Ben'

In [33]:
def dict_del(buckets, key):
    initial_hash = hash(key)
    size = len(buckets)
    for probe in probes(initial_hash, size):
        bucket = buckets[probe]
        if bucket is None:
            raise KeyError(key)
        elif bucket.key_hash == initial_hash and bucket.key == key:
            buckets[probe] = None
            break

In [34]:
toy_dict = [None] * 8
dict_set(toy_dict, 0, "Python")
dict_set(toy_dict, 1, "is awesome")
dict_set(toy_dict, 2, "!")
dict_set(toy_dict, 8, "Ben")
toy_dict

[Bucket(key_hash=0, key=0, value='Python'),
 Bucket(key_hash=1, key=1, value='is awesome'),
 Bucket(key_hash=2, key=2, value='!'),
 None,
 None,
 None,
 Bucket(key_hash=8, key=8, value='Ben'),
 None]

In [35]:
dict_del(toy_dict, 2)
toy_dict

[Bucket(key_hash=0, key=0, value='Python'),
 Bucket(key_hash=1, key=1, value='is awesome'),
 None,
 None,
 None,
 None,
 Bucket(key_hash=8, key=8, value='Ben'),
 None]

## Resizing

### What happens to a full dictionary?

In [36]:
for i, probe in enumerate(probes(hash("Learning Dicts"), 15)):
    if i > 15: break
    print(f"{probe = }")

probe = 2
probe = 6
probe = 6
probe = 12
probe = 0
probe = 6
probe = 10
probe = 0
probe = 10
probe = 10
probe = 8
probe = 2
probe = 2
probe = 10
probe = 2
probe = 10


Key takeaway - **Cannot have 100% full dictionaries**

### Load Factor - How big is our dictionary?

$$\text{load factor}(\alpha) = \frac{\text{number of used buckets}}{\text{number of available buckets}}$$

* Measures how many of the "buckets" in the dictionary are used
* Performance _usually_ goes down as load factor increases
* Resize if load factor becomes:
    * too small (to save space)
    * too large (to increase lookup performance)

* Python resizes when the load factor isn't between $\alpha_{\text{min}} = \frac{1}{3}$ and $\alpha_{\text{max}} = \frac{2}{3}$
* $\alpha_{\text{max}}$ is usual for HashMap implementation (usually $\approx 0.6$ to $\approx 0.75$)

In [37]:
def dict_load_factor(buckets):
    return sum(x is not None for x in buckets) / len(buckets)

In [38]:
dict_load_factor(toy_dict)

0.375

In [39]:
def dict_resize(buckets, grow):
    """
    Resize a dictionary to either double or half in size

    Modifies ``buckets`` inplace
    """
    new_size = len(buckets) << 1 if grow else len(buckets) >> 1
    # deepcopy can be avoided, but more complicated.
    old_buckets = deepcopy(buckets)
    buckets[:] = [None] * new_size

    for bucket in old_buckets:
        if bucket:
            dict_set(buckets, bucket.key, bucket.value)

In [40]:
toy_dict = [None] * 8
dict_set(toy_dict, 0, "Python")
dict_set(toy_dict, 1, "is awesome")
dict_set(toy_dict, 2, "!")
dict_set(toy_dict, 8, "Ben")
dict_resize(toy_dict, True)

### Why avoid resizing?

In [41]:
@dataclass
class Bucket:
    key_hash: int
    key: Hashable
    value: Any
    is_deleted: bool

In [42]:
def dict_set(buckets, key, value):
    initial_hash = hash(key)
    size = len(buckets)
    bucket = Bucket(initial_hash, key, value, False)
    for probe in probes(initial_hash, size):
        if buckets[probe] is None or bucket.is_deleted: 
            buckets[probe] = bucket
            break

In [43]:
def dict_get(buckets, key):
    initial_hash = hash(key)
    size = len(buckets)
    for probe in probes(initial_hash, size):
        bucket = buckets[probe]
        if bucket is None:
            raise KeyError(key)
        elif bucket.key_hash == initial_hash and bucket.key == key and not bucket.is_deleted:
            return bucket.value

In [44]:
def dict_del(buckets, key):
    initial_hash = hash(key)
    size = len(buckets)
    for probe in probes(initial_hash, size):
        bucket = buckets[probe]
        if bucket is None:
            raise KeyError(key)
        elif bucket.key_hash == initial_hash and bucket.key == key and not bucket.is_deleted:
            buckets[probe].is_deleted = True
            break

In [ ]:
toy_dict = [None] * 8
dict_set(toy_dict, 0, "Python")
dict_set(toy_dict, 1, "is awesome")
dict_set(toy_dict, 2, "!")
toy_dict

In [46]:
dict_get(toy_dict, 2)

'!'

In [47]:
dict_del(toy_dict, 2)

In [48]:
toy_dict

[Bucket(key_hash=0, key=0, value='Python', is_deleted=False),
 Bucket(key_hash=1, key=1, value='is awesome', is_deleted=False),
 Bucket(key_hash=2, key=2, value='!', is_deleted=True),
 None,
 None,
 None,
 None,
 None]

In [49]:
def is_valid_bucket(bucket):
    if bucket is None:
        return False
    return not bucket.is_deleted

def dict_load_factor(buckets):
    return sum(1 for x in buckets if is_valid_bucket(x)) / len(buckets)

In [50]:
def dict_resize(buckets, grow):
    """
    Resize a dictionary to either double or half in size

    Modifies ``buckets`` inplace
    """
    new_size = len(buckets) << 1 if grow else len(buckets) >> 1
    # deepcopy can be avoided, but more complicated.
    old_buckets = deepcopy(buckets)
    buckets[:] = [None] * new_size

    for bucket in old_buckets:
        if bucket and not bucket.is_deleted:
            dict_set(buckets, bucket.key, bucket.value)

In [51]:
dict_resize(toy_dict, True)

## How does Python do it differently?

# Improvements

## Designing a good hash function

* Up to this point we've simply used Python's inbuilt function hash to compute the hashes for our hash table
* How do we know if this is a good hash function?
* What does it even mean for a hash function to be good?
* Let's try to answer some of these questions!

### Properties we might want in a good hash function


1. Uniform Distribution
2. One-Way Functions
3. Fast

It turns out that designing a function with all of these properties is hard. We'll have to make compromises somewhere. They skill becomes knowing which compromises are right for your application.